# Setup

In [ ]:
!git clone "https://github.com/abdul2706/NuCLS.git"

In [ ]:
%cd /content/NuCLS

In [ ]:
!pwd

## Make Necessary Directories

In [ ]:
!mkdir "/content/NuCLS/data/tcga-nucleus/v4_2020-04-05_FINAL_CORE/CORE_SET/"
!mkdir "/content/NuCLS/data/tcga-nucleus/v4_2020-04-05_FINAL_CORE/CORE_SET/QC/"
!mkdir "/content/NuCLS/results/"
!mkdir "/content/NuCLS/results/tcga-nucleus/"
!mkdir "/content/NuCLS/results/tcga-nucleus/models/"
!mkdir "/content/NuCLS/results/tcga-nucleus/models/v4_2020-04-05_FINAL_CORE_QC/"

## Install Packages

In [ ]:
!pip install gitpython
!pip install histomicstk --find-links https://girder.github.io/large_image_wheels

# Download and Unzip Dataset

In [ ]:
!gdown --id "1k350VQeegN5hMxRK9Vpc65fdLe3wsqYy"

In [ ]:
!unzip "QC.zip" -d "/content/NuCLS/data/tcga-nucleus/v4_2020-04-05_FINAL_CORE/CORE_SET/" &> /dev/null

In [ ]:
!ls "/content/NuCLS/data/tcga-nucleus/v4_2020-04-05_FINAL_CORE/CORE_SET/QC/csv" -1 | wc -l
!ls "/content/NuCLS/data/tcga-nucleus/v4_2020-04-05_FINAL_CORE/CORE_SET/QC/mask" -1 | wc -l
!ls "/content/NuCLS/data/tcga-nucleus/v4_2020-04-05_FINAL_CORE/CORE_SET/QC/rgbs" -1 | wc -l
!ls "/content/NuCLS/data/tcga-nucleus/v4_2020-04-05_FINAL_CORE/CORE_SET/QC/rgbs_colorNormalized" -1 | wc -l
!ls "/content/NuCLS/data/tcga-nucleus/v4_2020-04-05_FINAL_CORE/CORE_SET/QC/train_test_splits" -1 | wc -l

In [ ]:
!rm -rf "QC.zip"

# Training

In [ ]:
import sys
import os
from os.path import join as opj
import argparse
from pprint import pprint

parser = argparse.ArgumentParser(description='Train nucleus model.')
parser.add_argument('-f', type=int, default=[1], nargs='+', help='fold(s) to run')
parser.add_argument('-g', type=int, default=[0], nargs='+', help='gpu(s) to use')
parser.add_argument('--qcd', type=int, default=1, help='use QCd data for training?')
parser.add_argument('--train', type=int, default=1, help='train?')
parser.add_argument('--vistest', type=int, default=1, help='visualize results on testing?')
args = parser.parse_args(['-f', '1', '-g', '0'])
args.qcd = bool(args.qcd)
args.train = bool(args.train)
args.vistest = bool(args.vistest)

# GPU allocation MUST happen before importing other modules
from GeneralUtils import save_configs, maybe_mkdir, AllocateGPU
# AllocateGPU(GPUs_to_use=args.g)

from nucls_model.MiscUtils import load_saved_otherwise_default_model_configs
from configs.nucleus_model_configs import CoreSetQC, CoreSetNoQC
from nucls_model.NucleusWorkflows import run_one_maskrcnn_fold

In [ ]:
# %%===========================================================================
# Configs

TAG = '[train.py]'
BASEPATH = os.path.realpath('.')
print(TAG, '[BASEPATH]', BASEPATH)
model_name = '002_MaskRCNN_tmp'
dataset_name = CoreSetQC.dataset_name if args.qcd else CoreSetNoQC.dataset_name
all_models_root = opj(BASEPATH, 'results', 'tcga-nucleus', 'models', f'{dataset_name}')
print(TAG, '[all_models_root]', all_models_root)
model_root = opj(all_models_root, model_name)
maybe_mkdir(model_root)

# load configs
configs_path = opj(model_root, 'nucleus_model_configs.py')
cfg = load_saved_otherwise_default_model_configs(configs_path=configs_path)

# for reproducibility, copy configs & most relevant code file to results
# if not os.path.exists(configs_path):
#     save_configs(configs_path=opj(BASEPATH, 'configs', 'nucleus_model_configs.py'), results_path=model_root)
# save_configs(configs_path=os.path.abspath(__file__), results_path=model_root, warn=False)
# save_configs(configs_path=opj(BASEPATH, 'nucls_model', 'NucleusWorkflows.py'), results_path=model_root, warn=False)

print(TAG, '[cfg]')
pprint(cfg)


In [ ]:
# %%===========================================================================
# Now run

for fold in args.f:
    run_one_maskrcnn_fold(
        fold=fold, cfg=cfg, model_root=model_root, model_name=model_name,
        qcd_training=args.qcd, train=args.train, vis_test=args.vistest)
